In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "/home/ylu/project")
from utils import *
import time

%load_ext autoreload
%autoreload 2

/home/ylu/anaconda3/envs/SPACEL/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import SPACEL
from SPACEL import Scube, Splane

In [3]:
data_folder = "./results/split_slices_CCF/"
results_folder = "./results/SPACEL/"
cache_folder = os.path.join(results_folder, 'Scube_outputs')
figures_folder = "./results/figures/SPACEL"
Path(results_folder).mkdir(parents=True, exist_ok=True)
Path(cache_folder).mkdir(parents=True, exist_ok=True)

In [4]:
import re
def list_and_sort_files_by_number(directory_path, suffix):
    def extract_number(filename):
        match = re.search(r'\d+', filename)
        return int(match.group()) if match else float('inf')
    try:
        files = [f for f in os.listdir(directory_path) if f.endswith(suffix)]
        sorted_files = sorted(files, key=extract_number)
        return sorted_files
    except FileNotFoundError:
        return f"The directory {directory_path} does not exist."
    except NotADirectoryError:
        return f"The path {directory_path} is not a directory."
    except PermissionError:
        return f"Permission denied to access {directory_path}."

directory_path = data_folder
sorted_files = list_and_sort_files_by_number(directory_path, '.h5ad')

In [5]:
## load the data
from tqdm import tqdm
slices = []
for f in tqdm(sorted_files):
    slices.append(ad.read_h5ad(os.path.join(data_folder, f)))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 129/129 [00:21<00:00,  6.01it/s]


In [6]:
spatial_key = "spatial_2D"
anno_key = "cell_type"

In [7]:
## Rotate the data
rotate_key = "spatial_rot"
rotations = np.load("./results/random_rotations.npy", allow_pickle=True)
for i in range(len(slices)):
    slices[i].obsm[rotate_key] = slices[i].obsm[spatial_key][:,:2].copy()
    mean = np.mean(slices[i].obsm[rotate_key], axis=0)
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] - mean
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] @ rotations[i].T + mean
    

In [8]:
spatial_key = "spatial_rot"
key_added = "aligned_spatial"

In [ ]:
## Perform the SPACEL with provided annotation alignment
sampling_num = 20000
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    slice1.obsm['spatial_2D'] = slice1.obsm[spatial_key]
    slice2.obsm['spatial_2D'] = slice2.obsm[spatial_key]
    slice1.obsm['spatial'] = slice1.obsm[spatial_key]
    slice2.obsm['spatial'] = slice2.obsm[spatial_key]
    align_slices = [slice1, slice2]
    time_start = time.time()
    Scube.align(
        align_slices,
        cluster_key=anno_key, 
        n_neighbors = 15, 
        n_threads=10,
        p=2,
        write_loc_path=os.path.join(results_folder, 'Scube_outputs/aligned_coordinates.csv')
    )
    align_slices[0].obsm[key_added] = align_slices[0].obsm['spatial_aligned'].values
    align_slices[1].obsm[key_added] = align_slices[1].obsm['spatial_aligned'].values
    time_end = time.time()
    
    R1, t1=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    R2, t2=solve_RT_by_correspondence(align_slices[0].obsm[spatial_key], align_slices[0].obsm[key_added])
    t = t1 @ R2.T + t2
    R = R2 @ R1
    alignment_results = {'R': R, 't': t, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                                                                                                                                                    | 0/128 [00:00<?, ?it/s]

Start alignment...
Alignment slice 1 to 0
Runtime: 116.85499405860901 s


  1%|█                                                                                                                                        | 1/128 [01:57<4:08:29, 117.40s/it]

Start alignment...
Alignment slice 1 to 0
Runtime: 140.50157165527344 s


  2%|██▏                                                                                                                                      | 2/128 [04:18<4:35:39, 131.27s/it]

...

Start alignment...
Alignment slice 1 to 0
Runtime: 176.7389748096466 s


 91%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎            | 116/128 [5:17:19<36:28, 182.39s/it]

Start alignment...
Alignment slice 1 to 0
Runtime: 187.96293687820435 s


 91%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍           | 117/128 [5:20:28<33:46, 184.26s/it]

Start alignment...
Alignment slice 1 to 0
